In [5]:
import pandas as pd
import numpy as np
import json
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_json('D:/wzy/bot/twi22/user.json')[['id', 'location']]
print(len(df))
df.head()

1000000


,id,location
0,u1217628182611927040,"Cambridge, MA"
1,u2664730894,🎈
2,u1266703520205549568,None
3,u1089159225148882949,"Charlottesville, VA"
4,u36741729,United States


In [3]:
followers = pd.read_csv('followers.csv')
following = pd.read_csv('following.csv')
label = pd.read_csv('twi22/label.csv')

In [6]:
# example: network in topic 3
tweet = pd.read_csv('topic_tweet3_climate.csv') 
tweet['uid'] = 'u'+tweet['uid'].astype(str)
select_user_id = set(tweet['uid'])&set(label['id'])
select_user_id = pd.DataFrame({'select_uid': list(select_user_id)})

# 只保留网络中讨论过这个话题的用户
select_followers = pd.merge(followers, select_user_id, left_on='source_id', right_on='select_uid', how='right')
select_followers.dropna(axis=0, inplace=True)
select_followers = pd.merge(select_followers, select_user_id, left_on='target_id', right_on='select_uid', how='right')
select_followers.dropna(axis=0, inplace=True)
select_followers = select_followers[['source_id', 'target_id']]

select_following = pd.merge(following, select_user_id, left_on='source_id', right_on='select_uid', how='right')
select_following.dropna(axis=0, inplace=True)
select_following = pd.merge(select_following, select_user_id, left_on='target_id', right_on='select_uid', how='right')
select_following.dropna(axis=0, inplace=True)
select_following = select_following[['source_id', 'target_id']]
# print(len(select_followers), len(select_following))

# 转化为(2, Num_edges)格式的numpy array, 其中第0行节点follow第1行节点
edges_1 = np.array([select_followers['target_id'].to_list(), select_followers['source_id'].to_list()])
edges_2 = np.array([select_following['source_id'].to_list(), select_following['target_id'].to_list()])
edges = np.concatenate([edges_1, edges_2], axis=1)
print("Check num of edges: ", edges.shape)

# 相应主题下有bot label和relation的uid（用于最终GNN training和effect estimatin）
final_uid = (set(select_user_id['select_uid'])&set(edges[0]))|(set(select_user_id['select_uid'])&set(edges[1]))
print("Num of user selected: ", len(final_uid))

Check num of edges:  (2, 412310)
Num of user selected:  63865


In [7]:
# Using networkx for network data analysis
import networkx as nx
G = nx.DiGraph()
# add the edges (directed)
for e in range(len(edges[0])):
    G.add_edge(edges[0][e], edges[1][e])
print("Num nodes: ", len(G.nodes), "Num edges: ", len(G.edges))

# add the node label (bot or human)
label.set_index('id', inplace=True)
for node_id in G.nodes():
    G.add_nodes_from([(node_id, {'label': label.loc[node_id, 'label']})])
print("Num nodes: ", len(G.nodes), "Num edges: ", len(G.edges))

Num nodes:  63865 Num edges:  407927
Num nodes:  63865 Num edges:  407927


In [8]:
df.set_index('id', inplace=True)
df.head()

,location
id,
u1217628182611927040,"Cambridge, MA"
u2664730894,🎈
u1266703520205549568,None
u1089159225148882949,"Charlottesville, VA"
u36741729,United States


In [34]:
with open('User_location.csv', 'r', encoding='utf-8', errors='ignore') as f:
    location_map1 = pd.read_csv(f)
mapping = dict(zip(location_map1['Original_location'], location_map1['map1']))
df['location_map1'] = df['location'].map(mapping)
print(df.head(20))

                                      location location_map1
id                                                          
u1217628182611927040             Cambridge, MA            UK
u2664730894                                  🎈           NaN
u1266703520205549568                      None           NaN
u1089159225148882949       Charlottesville, VA           USA
u36741729                        United States           USA
u1365527332627247104  Salem, Tamil Nadu, India           NaN
u1679822588                                 🇬🇧           NaN
u1519144464             Umhlanga, South Africa           NaN
u15211869              ÜT: 38.911326,-77.04508           NaN
u1309034737756000256                      None           NaN
u1341789703633178624                      None           NaN
u848975226                                None           NaN
u1109809482656149504                      None           NaN
u138814032                           Argentina     Argentina
u457554412              

In [23]:
G_HfollowingH = G.edge_subgraph([(u,v) for u,v,d in G.edges(data=True) if G.nodes[u]['label']=='human' and G.nodes[v]['label']=='human'])
G_HfollowingB = G.edge_subgraph([(u,v) for u,v,d in G.edges(data=True) if G.nodes[u]['label']=='human' and G.nodes[v]['label']=='bot'])
G_BfollowingH = G.edge_subgraph([(u,v) for u,v,d in G.edges(data=True) if G.nodes[u]['label']=='bot' and G.nodes[v]['label']=='human'])
G_BfollowingB = G.edge_subgraph([(u,v) for u,v,d in G.edges(data=True) if G.nodes[u]['label']=='bot' and G.nodes[v]['label']=='bot'])

In [73]:
loc_choice = ['UK', 'USA', 'Argentina', 'France', 'Nigeria', 'Germany', 'Iran', 
    'India', 'Global', 'Thailand', 'Indonesia', 'Netherlands',
    'Ireland', 'Kenya', 'Mexico', 'Japan', 'Egypt', 'Canada', 'Spain',
    'South Africa', 'Australia', 'HK', 'Pakistan', 'Denmark', 'China',
    'Uganda', 'Sweden', 'Singapore', 'Switzerland', 'Vague', 'Austria',
    'Europe', 'Tanzania', 'Malaysia', 'Israel', 'Bangladesh', 'Brasil',
    'Turkey', 'Ghana', 'Italy', 'New Zealand', 'Venezuela', 'Colombia',
    'Russia', 'Brazil', 'Belgium', 'Africa', 'UAE', 'Norway', 'Peru',
    'Sri Lanka', 'Metaverse', 'Portugal', 'Finland', 'Philippines',
    'Ukraine', 'Chile', 'Saudi Arabia']

In [78]:
def cal_homo_rate(Input_G):
    homo_rate = []
    for node in Input_G:
        location = df.loc[node]['location_map1']
        if location not in loc_choice:
            continue
        following_nodes = list(G.successors(node))
        if len(following_nodes)==0:
            continue
        sum_s = []
        for flw_node in following_nodes:
            location_flw = df.loc[flw_node]['location_map1']
            if location_flw not in loc_choice:
                continue
            else:
                sum_s.append((location_flw == location))
        if len(sum_s)==0:
            continue

        rate = sum(sum_s) / len(sum_s)
        homo_rate.append(rate)
    return homo_rate

def cal_neighomo_rate(Input_G):
    homo_rate = []
    for node in Input_G:
        location = df.loc[node]['location_map1']
        if location not in loc_choice:
            continue
        following_nodes = list(G.successors(node))
        if len(following_nodes)==0 or len(following_nodes)==1:
            continue

        sum_s = []
        for i in range(len(following_nodes)):
            location_i = df.loc[following_nodes[i]]['location_map1']
            if location_i not in loc_choice:
                continue
            for j in range(i, len(following_nodes)):
                location_j = df.loc[following_nodes[j]]['location_map1']
                if location_j not in loc_choice:
                    continue    
                else:
                    sum_s.append((location_i == location_j))
        if len(sum_s)==0:
            continue

        rate = sum(sum_s) / len(sum_s)
        homo_rate.append(rate)
    return homo_rate

hfh_homo = cal_homo_rate(G_HfollowingH)
hfb_homo = cal_homo_rate(G_HfollowingB)

hfh_neighomo = cal_neighomo_rate(G_HfollowingH)
bfh_neighomo = cal_neighomo_rate(G_BfollowingH)

print("Human following Human: ", np.mean(hfh_homo))
print("Human following Bot: ", np.mean(hfb_homo))

print("Human following Human: ", np.mean(hfh_neighomo))
print("Bot following Human: ", np.mean(bfh_neighomo))